In [13]:
import torch
import pandas as pd
import numpy as np
import torchio as tio
from torch.utils.data import DataLoader
import os
import SimpleITK as sitk
from zipfile import ZipFile
from zipfile import BadZipFile
import dask.dataframe as dd
import os
import multiprocessing as mp
import functools
from functools import partial
import Standardize
import Resampling
import utilsPreProcessing
from utilsPreProcessing import write_to_modif_path 
from registration.elastixRegister import reg_adc_hbv_to_t2w

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
# if using a Jupyter notebook, includue:
%matplotlib inline


## managment of files 
managment of files is done via managePicaiFiles.sh - create directories download and unpack files saves basic metadata and do simple metadata preprocessing
sh managePicaiFiles.sh

In [15]:
df = pd.read_csv('/home/sliceruser/data/metadata/processedMetaData.csv')
#currently We want only imagfes with associated masks
df = df.loc[df['isAnyMissing'] ==False]
df = df.loc[df['isAnythingInAnnotated']>0 ]
# ignore all with deficient spacing
for keyWord in ['t2w','adc', 'cor','hbv','sag'  ]:    
    colName=keyWord+ "_spac_x"
    df = df.loc[df[colName]>0 ]
#just for testing    
df=df.sample(n = 4)#TODO remove

df.to_csv('/home/sliceruser/data/metadata/processedMetaData_current.csv') 


## Standarization
primary preprocessing - removing ouliers, put values between 0 and 255 bias field correction Nyul standarization binarizing labels

In [3]:
import Standardize
import pandas as pd
trainedModelsBasicPath='/home/sliceruser/data/preprocess/standarizationModels'
for keyWord in ['t2w','adc', 'cor','hbv','sag'  ]:
    Standardize.iterateAndStandardize(keyWord,df,trainedModelsBasicPath)   
Standardize.iterateAndchangeLabelToOnes(df)


fitting normalizer  t2w
fitting normalizer  adc
fitting normalizer  cor
fitting normalizer  hbv
fitting normalizer  sag


In [ ]:
#Setting spacing of adc and HBV to t2w so then there would be less resampling needed during registration

In [ ]:
def resample_adc_hbv_to_t2w(row,secondCol ):
    pathT2w= row['t2w']
    pathh= row[secondCol] 
    newPath = pathh.replace(".mha","_resmaplA.mha" )
    #we check weather resampling was already done if not we do the resampling
    if(len(row[secondCol+'_resmaplA'] )<3):
        imageT2W = sitk.ReadImage(pathT2w)
        targetSpacing = imageT2W.GetSpacing()
        try:
            resampled = Resampling.resample_with_GAN(pathh,targetSpacing)
        except:
            print("error resampling")
        resampled = Resampling.resample_with_GAN(pathh,targetSpacing)

        write_to_modif_path(resampled,pathh,".mha","_resmaplA.mha" )
    return newPath

#needs to be on single thread as resampling GAN is acting on GPU
# we save the metadata to main pandas data frame 
df["adc_resmaplA"]=df.apply(lambda row : resample_adc_hbv_to_t2w(row, 'adc')   , axis = 1) 
df["hbv_resmaplA"]=df.apply(lambda row : resample_adc_hbv_to_t2w(row, 'hbv')   , axis = 1) 
df.to_csv('/home/sliceruser/data/metadata/processedMetaData.csv') 
        



origSize (84, 128, 19)
new_size (336, 512, 19)
1/1 [==============================] - 10s 10s/step
origSize (84, 128, 19)
new_size (336, 512, 19)
1/1 [==============================] - 10s 10s/step
origSize (84, 128, 19)
new_size (336, 512, 19)
1/1 [==============================] - 10s 10s/step
origSize (84, 128, 19)
new_size (336, 512, 19)
1/1 [==============================] - 10s 10s/step
origSize (84, 128, 19)
new_size (336, 512, 18)
1/1 [==============================] - 10s 10s/step
origSize (84, 128, 19)
new_size (336, 512, 18)
1/1 [==============================] - 10s 10s/step
origSize (256, 256, 27)
new_size (1023, 1023, 27)
1/1 [==============================] - 20s 20s/step


In [ ]:
#Registration of adc and hb

In [ ]:
elacticPath='/home/sliceruser/Slicer/NA-MIC/Extensions-30822/SlicerElastix/lib/Slicer-5.0/elastix'
reg_prop='/home/sliceruser/locTemp/piCaiCode/preprocessing/registration/parameters.txt'      
        
for keyWord in ['adc_resmaplA','hbv_resmaplA']:    
    with mp.Pool(processes = mp.cpu_count()) as pool:
        pool.map(partial(reg_adc_hbv_to_t2w,colName=keyWord,elacticPath=elacticPath,reg_prop=reg_prop )  ,list(df.iterrows()))    
 

/home/sliceruser/Slicer/NA-MIC/Extensions-30822/SlicerElastix/lib/Slicer-5.0/elastix -f /home/sliceruser/data/orig/10451/10451_1000459_t2w.mha -m /home/sliceruser/data/orig/10451/10451_1000459_adc.mha -out /home/sliceruser/data/orig/10451/10451_1000459_adc_for_adc -p /home/sliceruser/locTemp/piCaiCode/preprocessing/registration/parameters.txt
/home/sliceruser/Slicer/NA-MIC/Extensions-30822/SlicerElastix/lib/Slicer-5.0/elastix -f /home/sliceruser/data/orig/10804/10804_1000820_t2w.mha -m /home/sliceruser/data/orig/10804/10804_1000820_adc.mha -out /home/sliceruser/data/orig/10804/10804_1000820_adc_for_adc -p /home/sliceruser/locTemp/piCaiCode/preprocessing/registration/parameters.txt
/home/sliceruser/Slicer/NA-MIC/Extensions-30822/SlicerElastix/lib/Slicer-5.0/elastix -f /home/sliceruser/data/orig/11032/11032_1001052_t2w.mha -m /home/sliceruser/data/orig/11032/11032_1001052_adc.mha -out /home/sliceruser/data/orig/11032/11032_1001052_adc_for_adc -p /home/sliceruser/locTemp/piCaiCode/preproc

## Getting all of the spacings 

In [9]:
df = pd.read_csv('/home/sliceruser/data/metadata/processedMetaData.csv')

"""
looking through all valid spacings (if it si invalid it goes below 0)
and displaying minimal maximal and rounded mean spacing and median
in my case median and mean values are close - and using the median values will lead to a bit less interpolations later
"""
spacingDict={}
for keyWord in ['t2w','adc', 'cor','hbv','sag'  ]: 
    for addedKey in ['_spac_x','_spac_y','_spac_z']:   
        colName = keyWord+addedKey
        liist = list(filter(lambda it: it>0 ,df[colName].to_numpy() ))
        minn=np.min(liist)                
        maxx=np.max(liist)
        meanRounded = round((minn+maxx)/2,1)
        medianRounded = round(np.median(liist),1)
        spacingDict[colName]=(minn,maxx,meanRounded,medianRounded)

spacingDict



{'t2w_spac_x': (0.234375, 0.78125, 0.5, 0.5),
 't2w_spac_y': (0.234375, 0.78125, 0.5, 0.5),
 't2w_spac_z': (2.200000060372773, 5.000000066297424, 3.6, 3.0),
 'adc_spac_x': (0.859375, 2.5098040103912, 1.7, 2.0),
 'adc_spac_y': (0.859375, 2.5098040103912, 1.7, 2.0),
 'adc_spac_z': (2.999998832062665, 5.000000232855622, 4.0, 3.0),
 'cor_spac_x': (0.286756128073, 0.9765625, 0.6, 0.6),
 'cor_spac_y': (0.286756128073, 0.9765625, 0.6, 0.6),
 'cor_spac_z': (0.9999999999999636, 4.49999998188102, 2.7, 3.0),
 'hbv_spac_x': (0.859375, 2.5098040103912, 1.7, 2.0),
 'hbv_spac_y': (0.859375, 2.5098040103912, 1.7, 2.0),
 'hbv_spac_z': (2.999998832062665, 5.000000232855622, 4.0, 3.0),
 'sag_spac_x': (0.254629641771, 0.78125, 0.5, 0.6),
 'sag_spac_y': (0.254629641771, 0.78125, 0.5, 0.6),
 'sag_spac_z': (2.999999882111563, 4.000404550206146, 3.5, 3.6)}

Now idea is to iteratively transform images into uniform spacing here i will choose median of the data as the target value and resample images to it to get uniform spacing
To be more precise we will look at median t2w spacing and adjust t2w, adc and hbv to it

In [ ]:


targetSpacingg=(spacingDict['t2w_spac_x'][3],spacingDict['t2w_spac_y'][3],spacingDict['t2w_spac_z'][3])
    
    
def resample_registered_to_given(row,colname,targetSpacing):
    path=row[colname]
    outPath = path.replace(".mha","_for_"+colName)
    registeredPath = outPath+"/result.0.mha"
    newPath = path.replace(".mha","_medianSpac.mha" )   
    try:
        resampled = Resampling.resample_with_GAN(registeredPath,targetSpacing)
    except:
        print("error resampling")
    resampled = Resampling.resample_with_GAN(registeredPath,targetSpacing)

    write_to_modif_path(resampled,path,".mha","_medianSpac.mha" )
    return newPath

"""
registered images were already resampled now time for t2w and labels
"""
def resample_t2w(row,targetSpacing):
    path=row['t2w']
    newPath = path.replace(".mha","_medianSpac.mha" )   
    try:
        resampled = Resampling.resample_with_GAN(registeredPath,targetSpacing)
    except:
        print("error resampling")
    resampled = Resampling.resample_with_GAN(registeredPath,targetSpacing)

    write_to_modif_path(resampled,path,".mha","_medianSpac.mha" )
    return newPath    

def resample_labels(row,targetSpacing):
    path=row['reSampledPath']
    newPath = path.replace(".mha","_medianSpac.mha" )   
    try:
        resampled = Resampling.resample_label_with_GAN(registeredPath,targetSpacing)
    except:
        print("error resampling")
    resampled = Resampling.resample_label_with_GAN(registeredPath,targetSpacing)

    write_to_modif_path(resampled,path,".mha","_medianSpac.mha" )
    return newPath        
    

#needs to be on single thread as resampling GAN is acting on GPU
# we save the metadata to main pandas data frame 
df["adc_med_spac"]=df.apply(lambda row : resample_registered_to_given(row, 'adc_resmaplA',targetSpacingg)   , axis = 1) 
df["hbv_med_spac"]=df.apply(lambda row : resample_registered_to_given(row, 'hbv_resmaplA',targetSpacingg)   , axis = 1) 
df["t2w_med_spac"]=df.apply(lambda row : resample_t2w(row,targetSpacingg)   , axis = 1) 
df["label_med_spac"]=df.apply(lambda row : resample_labels(row,targetSpacingg)   , axis = 1) 


df.to_csv('/home/sliceruser/data/metadata/processedMetaData.csv') 
        

In [ ]:
#Now we need to retrieve the maximum dimensions of resampled images

In [ ]:
def get_spatial_meta(row,colName):
    #row=row[1]
    patId=str(row['patient_id'])
    path=str(row[colName])
    if(len(path)>1):
        image = sitk.ReadImage(path)
        sizz= ifShortReturnMinus(image.GetSize(),patId,colName )
        spac= ifShortReturnMinus(image.GetSpacing(),patId,colName)
        orig= ifShortReturnMinus(image.GetOrigin(),patId,colName)
        return list(sizz)+list(spac)+list(orig)
    return [-1,-1,-1,-1,-1,-1,-1,-1,-1]
for keyWord in ['t2w_med_spac']:    
    resList=[]
    with mp.Pool(processes = mp.cpu_count()) as pool:
        resList=pool.map(partial(get_spatial_meta,colName=keyWord)  ,list(df.iterrows()))    
    print(type(resList))    
    df[keyWord+'_sizz_x']= list(map(lambda arr:arr[0], resList))    
    df[keyWord+'_sizz_y']= list(map(lambda arr:arr[1], resList))    
    df[keyWord+'_sizz_z']= list(map(lambda arr:arr[2], resList))
    
df.to_csv('/home/sliceruser/data/metadata/processedMetaData.csv') 

#getting maximum size - so one can pad to uniform size if needed (for example in validetion test set)
median_spac_max_size_x = np.max(list(filter(lambda it: it>0 ,df['t2w_med_spac_sizz_x'].to_numpy() )))
median_spac_max_size_y = np.max(list(filter(lambda it: it>0 ,df['t2w_med_spac_sizz_y'].to_numpy() )))
median_spac_max_size_z = np.max(list(filter(lambda it: it>0 ,df['t2w_med_spac_sizz_z'].to_numpy() )))


maxSize = (median_spac_max_size_x,median_spac_max_size_y,median_spac_max_size_z  )
maxSize

In [ ]:
#We will also experiment with some other spacing values so we will be able to set it as hyperparameter and check wheather for example low resolution will give better result